In [ ]:
from source_files import SNOW_DEPTH_DIR, CASI_COLORS
from plot_helpers import *

from raster_compare.plots import PlotBase
from raster_compare.base import RasterFile

import math

from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection

In [ ]:
aso_snow_depth = RasterFile(
    SNOW_DEPTH_DIR / '1m/20180524_ASO_snow_depth_1m.tif',
    band_number=1
)
aso_snow_depth_values = aso_snow_depth.band_values()
np.ma.masked_where(
    aso_snow_depth_values <= 0.0,
    aso_snow_depth_values,
    copy=False
)

sfm_snow_depth = RasterFile(
    SNOW_DEPTH_DIR / '1m/20180524_Agisoft_snow_depth_1m.tif',
    band_number=1
)
assert aso_snow_depth.geo_transform == sfm_snow_depth.geo_transform

sfm_snow_depth_values = sfm_snow_depth.band_values()
np.ma.masked_where(
    aso_snow_depth_values.mask,
    sfm_snow_depth_values,
    copy=False
)

HS_SNOW_ON = SNOW_DEPTH_DIR / 'hillshade/20180524_Lidar_hs_ERW_basin_3m.tif'
hillshade_snow_on = RasterFile(HS_SNOW_ON, band_number=1);

# Snow Depth Difference

In [ ]:
HIST_BIN_WIDTH = .25
bins = np.concatenate((
    np.arange(0, 2. + HIST_BIN_WIDTH, HIST_BIN_WIDTH),
    [math.ceil(np.nanmax(sfm_snow_depth_values))],
))

In [ ]:
AREA_PLOT_OPTS = dict(
    nrows=1, ncols=2, sharex=True, sharey=True, figsize=(10,8), dpi=750
)

COLORBAR_POSITION = dict(
    right=0.90, rect=[0.91, 0.217, 0.02, 0.568],
)

SFM_UNDER = dict(color='indigo', alpha=0.4)
BLUE_CMAP.set_under(**SFM_UNDER)

imshow_opts = dict(
    extent=sfm_snow_depth.extent,
    norm=colors.BoundaryNorm(
        boundaries=bins, ncolors=BLUE_CMAP.N,
    ),
    cmap=BLUE_CMAP,
)

In [ ]:
def area_plot():
    fig, axes = plt.subplots(**AREA_PLOT_OPTS)

    for ax in axes:
        ax.imshow(
            hillshade_snow_on.band_values(),
            extent=sfm_snow_depth.extent,
            cmap='gray', clim=(1, 255), alpha=0.25,
        )
        ax.tick_params(axis='both', direction='inout', size=5)
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        
        ax.set_facecolor('whitesmoke')

    return fig, axes

In [ ]:
fig, (ax1, ax2) = area_plot()

ax1.imshow(
    sfm_snow_depth_values,
    **imshow_opts
)
    
ax1.annotate('a)', xy=(335600, 4306300), fontsize=14)
ax1.add_artist(mpatches.Circle((323000, 4319750), 250, **SFM_UNDER))
ax1.annotate('No SfM snow depth', xy=(323400, 4319950), fontsize=LABEL_SIZE)
ax1.add_artist(
    ScaleBar(1.0, location='lower left', pad=0.5, scale_loc='top', box_color='none')
)
ax1.annotate(
    'N', size=LABEL_SIZE, 
    xy=(325500, 4320050), xytext=(325500, 4322000), 
    ha="center", va="center", 
    arrowprops=dict(arrowstyle="wedge,tail_width=1.25", facecolor='black')
)

im_data = ax2.imshow(
    aso_snow_depth_values,
    **imshow_opts,
)
ax2.annotate('b)', xy=(335600, 4306300), fontsize=14)
ax2.add_patch(
    Rectangle((327200, 4320470 - 13879), 1000, 1000,  ec='darkred', ls='--', fill=False),
)
ax2.legend(handles=[
    mlines.Line2D([0], [0], label='Figure 4', color='darkred', ls='--'),
], loc='lower left', facecolor='none', edgecolor='none')
   
matplotlib.rcParams["ytick.major.pad"] = 2
PlotBase.insert_colorbar(
    ax2, 
    im_data,
    SNOW_DEPTH_LABEL,
    ticks=[0, .5, 1, 1.5, 2],
    labelpad=12,
    **COLORBAR_POSITION
)

del im_data

## Zoom comparison

In [ ]:
rgb_zoom_tif = plt.imread((SNOW_DEPTH_DIR / 'ERW_zoom/ERW_zoom.tif').as_posix())

casi_zoom_values = RasterFile(
    SNOW_DEPTH_DIR / '1m/20180524_ASO_CASI_ERW_basin_1m_zoom.vrt',
    band_number=1
).band_values()

In [ ]:
matplotlib.rcParams['axes.titlesize'] = LABEL_SIZE
matplotlib.rcParams['axes.titlepad'] = 4
matplotlib.rcParams['axes.labelsize'] = LABEL_SIZE + 2
matplotlib.rcParams['axes.labelpad'] = 0
matplotlib.rcParams['xtick.labelsize'] = 0
matplotlib.rcParams['ytick.labelsize'] = 0

In [ ]:
fig = plt.figure(dpi=500, figsize=(9, 4.7))
grid_spec = fig.add_gridspec(
    nrows=2, ncols=2, wspace=0.11, hspace=0.12
)

for cell in range(3):
    cell_grid = grid_spec[cell].subgridspec(1, 2, wspace=0.05)
    ax_1_1 = fig.add_subplot(cell_grid[0, 0])
    ax_1_2 = fig.add_subplot(cell_grid[0, 1:])

    if cell == 0:
        resolution = 1
    elif cell == 1:
        resolution = 3
    elif cell == 2:
        resolution = 50
    
    aso_snow_depth_zoom = RasterFile(
        SNOW_DEPTH_DIR / f'{resolution}m/20180524_ASO_snow_depth_{resolution}m_zoom.vrt',
        band_number=1
    )
    aso_snow_depth_zoom_values = aso_snow_depth_zoom.band_values()
    aso_snow_depth_zoom_values = np.ma.masked_where(
        aso_snow_depth_zoom_values <= 0.0,
        aso_snow_depth_zoom_values,
        copy=False
    )

    sfm_snow_depth_zoom = RasterFile(
        SNOW_DEPTH_DIR / f'{resolution}m/20180524_Agisoft_snow_depth_{resolution}m_zoom.vrt',
        band_number=1
    )
    sfm_snow_depth_zoom_values = np.ma.masked_where(
        aso_snow_depth_zoom_values.mask,
        sfm_snow_depth_zoom.band_values(),
        copy=False
    )

    imshow_opts['extent'] = sfm_snow_depth_zoom.extent
    
    ax_1_1.imshow(
        sfm_snow_depth_zoom_values,
        **imshow_opts
    )
    ax_1_1.set_title('SfM')
    ax_1_1.set_ylabel(f' {resolution}m Resolution')
    
    ax_1_2.imshow(
        aso_snow_depth_zoom_values,
        **imshow_opts
    )
    ax_1_2.set_title('ASO')


cell_4 = grid_spec[3].subgridspec(1, 2)
ax_4_1 = fig.add_subplot(cell_4[0, 0])
ax_4_2 = fig.add_subplot(cell_4[0, 1:])

ax_4_1.imshow(
    casi_zoom_values, 
    extent=sfm_snow_depth_zoom.extent,
    cmap=colors.ListedColormap(CASI_COLORS),
    alpha=0.8,
)
ax_4_1.set_title('Classification')

ax_4_2.imshow(
    rgb_zoom_tif,
    extent=sfm_snow_depth_zoom.extent,
)
ax_4_2.set_title('Orthomosaic')

for ax in fig.get_axes():
    ax.tick_params(axis='both', direction='inout', size=5)
    ax.set_xticklabels([])
    ax.set_yticklabels([])


## CASI Classification 

In [ ]:
classifier_data = load_classifier_data(aso_snow_depth_values.mask)
sd_difference_values = sfm_snow_depth_values - aso_snow_depth_values

np.ma.masked_where(
    sfm_snow_depth_values <= 0.0,
    sd_difference_values,
    copy=False
)
classification_plot = np.ma.masked_where(
    sfm_snow_depth_values <= 0.0,
    classifier_data,
).astype(np.int8);

In [ ]:
bins = np.concatenate((
    [math.floor(np.nanmin(sd_difference_values))],
    np.arange(-2., 2. + HIST_BIN_WIDTH, HIST_BIN_WIDTH),
    [math.ceil(np.nanmax(sd_difference_values))],
))
imshow_opts = dict(
    extent=sfm_snow_depth.extent,
    norm=colors.BoundaryNorm(
        boundaries=bins, ncolors=RED_BLUE_CMAP.N,
    ),
    cmap=RED_BLUE_CMAP,
)

In [ ]:
fig, (ax1, ax2) = area_plot()
fig.subplots_adjust(wspace=.15)

im_data = ax1.imshow(
    sd_difference_values,     
    **imshow_opts
)
ax1.set_title("Snow Depth Differences")
PlotBase.insert_colorbar(ax1, im_data, 'Snow Depth Difference (m)')

im_data = ax2.imshow(
    classification_plot, 
    extent=sfm_snow_depth.extent,
    cmap=colors.ListedColormap(CASI_COLORS),
    alpha=0.8,
)
ax2.set_title("Snow Depth Differences - Classification")
PlotBase.insert_colorbar(ax2, im_data, 'Classification');

In [ ]:
dem_values = load_reference_dem(aso_snow_depth_values.mask)

In [ ]:
high_elevation = np.ma.masked_where(
    dem_values <= 3500,
    sd_difference_values,
)
low_elevation = np.ma.masked_where(
    dem_values > 3500,
    sd_difference_values,
)

fig, (ax1, ax2) = area_plot()

ax1.imshow(
    high_elevation, 
    **imshow_opts
)
ax1.set_title("Snow Depth Differences - High Elevation >= 3500m")

im_data = ax2.imshow(
    low_elevation, 
    **imshow_opts,
)
ax2.set_title("Snow Depth Differences - Low Elevation < 3500")

PlotBase.insert_colorbar(ax2, im_data, 'Snow Depth Difference (m)', **COLORBAR_POSITION);

In [ ]:
high_elevation = np.ma.masked_where(
    np.ma.masked_outside(dem_values, 3100, 3500).mask,
    sd_difference_values,
)
low_elevation = np.ma.masked_where(
    np.ma.masked_outside(dem_values, 3800, 3900).mask,
    sd_difference_values,
)

fig, (ax1, ax2) = area_plot()
ax1.imshow(
    high_elevation, 
    **imshow_opts
)
ax1.set_title("Snow Depth Differences - 3100m < Elevation < 3200m")

im_data = ax2.imshow(
    low_elevation, 
    **imshow_opts,
)
ax2.set_title("Snow Depth Differences - 3800 < Elevation < 3900")

PlotBase.insert_colorbar(ax2, im_data, r'$\Delta$ Snow Depth (m)', **COLORBAR_POSITION);

In [ ]:
fig, (ax1, ax2) = area_plot()

ax1.imshow(
    sd_difference_values, 
    **imshow_opts
)
ax1.set_title("Snow Depth Differences")

im_data = ax2.imshow(
    sfm_snow_free_values - dem_values, 
    **imshow_opts,
)
ax2.set_title("Snow Free - Reference DEM")

PlotBase.insert_colorbar(ax2, im_data, **COLORBAR_POSITION);

In [ ]:
data = [
    {
        'data': sfm_snow_free_values - dem_values,
        'label': 'SfM snow free - DEM',
        'color': 'dodgerblue',
    },
]

with plot_histogram(data, (-6, 6), figsize=(10, 8)) as ax:
    ax.set_title('Elevation Differences (SFM snow free - Reference DEM)');